In [54]:
%pip install nltk
%pip install lxml
%pip install -U scikit-learn


import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import email
from bs4 import BeautifulSoup


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Load the dataset
df = pd.read_csv('data.csv')

def read_email_from_string(s):
    message = email.message_from_string(s)
    return message

def extract_email_body(message):
    if message.is_multipart():
        for part in message.walk():
            type_content = part.get_content_maintype()
            if type_content == 'text':
                message = part
                break
        else:
            return ''
    body = message.get_payload(decode=False)
    return body

def remove_html(s):
    soup = BeautifulSoup(s, 'lxml')
    for sp in soup(['script', 'style', 'head', 'meta', 'noscript', 'http']):
        sp.decompose()
    s = ' '.join(soup.stripped_strings)
    return s

# Function to preprocess text
def preprocess_text(text):
    # Extract email body
    body = extract_email_body(read_email_from_string(text))
    body = remove_html(body)

    # Tokenize the text
    words = nltk.word_tokenize(body)

    # Remove punctuation and convert to lower case
    words = [word.lower() for word in words if word.isalpha()]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    words = [word for word in words if word != 'subject']
    return ' '.join(words)

df['origin'] = df['origin'].apply(preprocess_text)

# Save the refactored emails back to the same file
df.to_csv('preprocessed.csv', index=False)
df.head()


C:\Users\Martin\AppData\Local\Temp\ipykernel_18528\990443069.py:21: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(s, 'lxml')


,label,origin
0,0,wrong bill grace forward original message rodr...
1,0,continued hilcorp old ocean deal dan hyvl writ...
2,0,several related issue resulted increase level ...
3,0,one year rate one mm volume greater mm day pri...
4,0,attached weekly deal report lex carroll enron ...


In [ ]:
df = df.sample(frac = 1, random_state = 1)
df = df.reset_index(drop = True)

# Create split index for 70/30 split
split_index = int(len(df) * 0.7)

# Split data
train_df, test_df = df[:split_index], df[split_index:]

validation_split_index = int(len(train_df) * 0.8)
train_df, val_df = train_df[:validation_split_index], df[validation_split_index:]


df = train_df.reset_index(drop = True)
df = test_df.reset_index(drop = True)


10111

In [ ]:

print(len(train_df))
print(len(val_df))
print(len(test_df))

18872
14830
10111


In [ ]:
# Count tokens in training set
token_counter = {}
for message in train_df['origin']:
  words = nltk.word_tokenize(message)

  for token in words:
    if token in token_counter:
      token_counter[token] += 1
    else:
      token_counter[token] = 1

token_counter

In [ ]:
def keep_token(proccessed_token, threshold):
  if proccessed_token not in token_counter:
    return False
  else:
    # Add condition to check length of token
    return token_counter[proccessed_token] > threshold and len(proccessed_token) > 3


In [ ]:
# for the current dataset 500 is a good value which generates a
# set with an acceptable length to be considered a feature set for the
# machine learning algorithms

# this is the Bag of Words approach

features = set()

for token in token_counter:
  if keep_token(token, 6000):
    features.add(token)

features = list(features)
features

In [ ]:
token_to_index_mapping = {t:i for t, i in zip(features, range(len(features)))}
token_to_index_mapping

In [ ]:
import numpy as np

def message_to_count_vector(message):
  count_vector = np.zeros(len(features))

  processed_list_of_tokens = nltk.word_tokenize(message)

  for token in processed_list_of_tokens:
    if token not in features:
      continue
    index = token_to_index_mapping[token]
    count_vector[index] += 1

  return count_vector

In [ ]:
message_to_count_vector(train_df['origin'].iloc[9010])

In [ ]:
# 1 spam; 0 not spam
train_df.iloc[9010]

In [ ]:
def extract_features_and_labels(dataframe):
  # Extract labels and convert to integer type
  labels = dataframe['label'].values.astype(int)

  # Extract messages
  messages = dataframe['origin']
  vector_counts = []

  # Convert each message to a count vector
  for msg in messages:
    vector = message_to_count_vector(msg)
    vector_counts.append(vector)

  # Convert list of count vectors to a numpy array and cast to integer type
  feature_matrix = np.asarray(vector_counts).astype(int)

  return feature_matrix, labels


In [ ]:
X_train_BOW, Y_train_BOW = extract_features_and_labels(train_df)

X_val_BOW, Y_val_BOW = extract_features_and_labels(val_df)

X_test_BOW, Y_test_BOW = extract_features_and_labels(test_df)




X_train_BOW.shape, Y_train_BOW.shape, X_test_BOW.shape, Y_test_BOW.shape

((18872, 13), (18872,), (10111, 13), (10111,))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(X_train_BOW)

X_train_BOW, X_test_BOW, X_val_BOW = scaler.transform(X_train_BOW), scaler.transform(X_test_BOW), scaler.transform(X_val_BOW)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

hyperparameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2']
}

X_train_val_BOW = np.concatenate((X_train_BOW, X_val_BOW), axis=0)
Y_train_val_BOW = np.concatenate((Y_train_BOW, Y_val_BOW), axis=0)

lrBOW = LogisticRegression(solver='liblinear', max_iter=30000)
grid_search = GridSearchCV(lrBOW, hyperparameters, cv=5)
best_model = grid_search.fit(X_train_val_BOW, Y_train_val_BOW)
best_params = best_model.best_params_
print('Best Parameters:', best_params)
print('Best Score:', best_model.best_score_)
X_train_BOW, X_val_BOW, Y_train_BOW, Y_val_BOW = train_test_split(X_train_val_BOW, Y_train_val_BOW, test_size=0.2, random_state=42)


# Train the model with the best parameters
lrBOW = LogisticRegression(solver='liblinear', max_iter=30000, **best_params)
lrBOW.fit(X_train_BOW, Y_train_BOW)
print(classification_report(Y_val_BOW, lrBOW.predict(X_val_BOW)))


Best Parameters: {'C': 100, 'penalty': 'l2'}
Best Score: 0.7590648174926719
              precision    recall  f1-score   support

           0       0.82      0.66      0.74      3330
           1       0.72      0.86      0.79      3411

    accuracy                           0.76      6741
   macro avg       0.77      0.76      0.76      6741
weighted avg       0.77      0.76      0.76      6741



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [55]:
def extract_features_and_labels_tfidf(dataframe):
  # Extract labels and convert to integer type
  labels = dataframe['label'].values.astype(int)

  # Extract messages
  messages = dataframe['origin']

  # Use TFIDF vectorizer to convert messages to TFIDF vectors
  vectorizer = TfidfVectorizer()
  tfidf_matrix = vectorizer.fit_transform(messages)

  return tfidf_matrix, labels

In [58]:
X_train_TFIDF, Y_train_TFIDF = extract_features_and_labels_tfidf(train_df)

X_val_TFIDF, Y_val_TFIDF = extract_features_and_labels_tfidf(val_df)

X_test_TFIDF, Y_test_TFIDF = extract_features_and_labels_tfidf(test_df)

print(X_train_TFIDF.shape)
print(X_val_TFIDF.shape)
print(X_test_TFIDF.shape)

(18872, 103048)
(14830, 89592)
(10111, 73849)


In [57]:
X_train_val_TFIDF = np.concatenate((X_train_TFIDF, X_val_TFIDF), axis=0)
Y_train_val_TFIDF = np.concatenate((Y_train_TFIDF, Y_val_TFIDF), axis=0)

lrTFIDF = LogisticRegression(solver='liblinear', max_iter=30000)

grid_search = GridSearchCV(lrTFIDF, hyperparameters, cv=5)

best_model = grid_search.fit(X_train_val_TFIDF, Y_train_val_TFIDF)

# Get the best parameters
best_params = best_model.best_params_
print('Best Parameters:', best_params)
print('Best Score:', best_model.best_score_)

# Split the sets again
X_train_TFIDF, X_val_TFIDF, Y_train_TFIDF, Y_val_TFIDF = train_test_split(X_train_val_TFIDF, Y_train_val_TFIDF, test_size=0.2, random_state=42)

lrTFIDF = LogisticRegression(solver='liblinear', max_iter=30000, **best_params)
lrTFIDF.fit(X_train_TFIDF, Y_train_TFIDF)

print(classification_report(Y_val_TFIDF, lrTFIDF.predict(X_val_TFIDF)))

ValueError: zero-dimensional arrays cannot be concatenated

In [ ]:
lrTFIDF = LogisticRegression()

lrTFIDF.fit(X_train_features, Y_train_TFIDF)

prediction_training_data = lrTFIDF.predict(X_train_features)

accuracy_training_data = accuracy_score(Y_train_TFIDF, prediction_training_data)

print('accuracy on training data: ', accuracy_training_data)

In [ ]:
prediction_test_data = lrTFIDF.predict(X_test_features)
accuracy_test_data = accuracy_score(Y_test_TFIDF, prediction_test_data)
print('accuracy on training data: ', accuracy_test_data)

In [ ]:
input_email = ["You have 18 unread notifications to review"]
input_data_features = feature_extraciton.transform(input_email)

prediction = lrTFIDF.predict(input_data_features)

print(prediction)

if(prediction[0]==1):
    print('Spam mail')

else:
    print('Ham mail')
